# **Performing text summarization with BART**

In [ ]:
!pip install torch==1.4.0. 


     |████████████████████████████████| 753.4MB 18kB/s 
ERROR: torchvision 0.9.1+cu101 has requirement torch==1.8.1, but you'll have torch 1.4.0 which is incompatible.
ERROR: torchtext 0.9.1 has requirement torch==1.8.1, but you'll have torch 1.4.0 which is incompatible.
  Found existing installation: torch 1.8.1+cu101
    Uninstalling torch-1.8.1+cu101:
      Successfully uninstalled torch-1.8.1+cu101


In [ ]:
%%capture
!pip install transformers==3.5.1

In [ ]:
from transformers import BartTokenizer

In [ ]:
from transformers import BartForConditionalGeneration

In [ ]:
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')

In [ ]:
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')

In [ ]:
text = """Machine learning (ML) is the study of computer algorithms that improve automatically through experience.It is seen as a subset of artificial intelligence. Machine learning algorithms build a mathematical model based on sample data, known as training data, in order to make predictions or decisions without being explicitly programmed to do so.Machine learning algorithms are used in a wide variety of applications, such as email filtering and computer vision, where it is difficult or infeasible to develop conventional algorithms to perform the needed tasks."""

In [ ]:
inputs = tokenizer([text], max_length=1024, return_tensors='pt')

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [ ]:
summary_ids = model.generate(inputs['input_ids'], num_beams=4, max_length=100, early_stopping=True)

In [ ]:
summary = ([tokenizer.decode(i, skip_special_tokens=True, clean_up_tokenization_spaces=False) for i in summary_ids])

In [ ]:
print(summary)

['Machine learning is the study of computer algorithms that improve automatically through experience. It is seen as a subset of artificial intelligence. Machine learning algorithms are used in a wide variety of applications, such as email filtering and computer vision, where it is difficult or infeasible to develop conventional algorithms to perform the needed tasks.']


# **Sentiment Analysis using Ktrain**

In [ ]:
!pip install ktrain==0.25.3

     |████████████████████████████████| 25.3MB 1.6MB/s 
     |████████████████████████████████| 6.8MB 25.6MB/s 
     |████████████████████████████████| 983kB 27.1MB/s 
     |████████████████████████████████| 266kB 39.4MB/s 
     |████████████████████████████████| 471kB 49.6MB/s 
  Created wheel for ktrain: filename=ktrain-0.25.3-cp37-none-any.whl size=25276312 sha256=2f9355d25e0c5893f534cb53a0905875f18c775aa3f88d1f5f90b63ceb9a3f34
  Stored in directory: /root/.cache/pip/wheels/86/cd/9c/ad53b98a3eccf4ddeddccfce99d353781f206a1ac4287ca721
  Created wheel for langdetect: filename=langdetect-1.0.9-cp37-none-any.whl size=993242 sha256=b209fe21b3d3b97f2fe420abcf32df418f41b3526b637f4b8bd8757ded370cb8
  Stored in directory: /root/.cache/pip/wheels/7e/18/13/038c34057808931c7ddc6c92d3aa015cf1a498df5a70268996
  Created wheel for syntok: filename=syntok-1.3.1-cp37-none-any.whl size=20919 sha256=01ca9c04ef233e127e23ba376196122056146f0a6b7b2c1e90ccb429adf5d80b
  Stored in directory: /root/.cache/pip/

In [ ]:
import ktrain
from ktrain import text 
import pandas as pd

Downloading and Load DataSet

In [ ]:
!gdown https://drive.google.com/uc?id=1-8urBLVtFuuvAVHi0s000e7r0KPUgt9f

Downloading...
From: https://drive.google.com/uc?id=1-8urBLVtFuuvAVHi0s000e7r0KPUgt9f
To: /content/reviews_Digital_Music_5.json
89.0MB [00:00, 124MB/s]


In [ ]:
df = pd.read_json(r'reviews_Digital_Music_5.json',lines=True)

**Let us have a look at a few rows of our dataset**

In [ ]:
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A3EBHHCZO6V2A4,5555991584,"Amaranth ""music fan""","[3, 3]","It's hard to believe ""Memory of Trees"" came ou...",5,Enya's last great album,1158019200,"09 12, 2006"
1,AZPWAXJG9OJXV,5555991584,bethtexas,"[0, 0]","A clasically-styled and introverted album, Mem...",5,Enya at her most elegant,991526400,"06 3, 2001"
2,A38IRL0X2T4DPF,5555991584,bob turnley,"[2, 2]",I never thought Enya would reach the sublime h...,5,The best so far,1058140800,"07 14, 2003"
3,A22IK3I6U76GX0,5555991584,Calle,"[1, 1]",This is the third review of an irish album I w...,5,Ireland produces good music.,957312000,"05 3, 2000"
4,A1AISPOIIHTHXX,5555991584,"Cloud ""...""","[1, 1]","Enya, despite being a successful recording art...",4,4.5; music to dream to,1200528000,"01 17, 2008"


In [ ]:
df = df[['reviewText', 'overall']]

In [ ]:
df.head()

,reviewText,overall
0,"It's hard to believe ""Memory of Trees"" came ou...",5
1,"A clasically-styled and introverted album, Mem...",5
2,I never thought Enya would reach the sublime h...,5
3,This is the third review of an irish album I w...,5
4,"Enya, despite being a successful recording art...",4


In [ ]:
df.describe()

,reviewText,sentiment
count,5000,5000
unique,4999,2
top,"Maybe it was his time in prison, or maybe it w...",positive
freq,2,4439


In [ ]:
sentiment = {1:'negative', 2:'negative', 3:'negative', 4:'positive', 5:'positive'}

In [ ]:
df['sentiment'] = df['overall'].map(sentiment)

**Now let us subset the dataset with only reviewText, and sentiment column as shown below:**

In [ ]:
df = df[['reviewText', 'sentiment']]

In [ ]:
df.head()

,reviewText,sentiment
0,"It's hard to believe ""Memory of Trees"" came ou...",positive
1,"A clasically-styled and introverted album, Mem...",positive
2,I never thought Enya would reach the sublime h...,positive
3,This is the third review of an irish album I w...,positive
4,"Enya, despite being a successful recording art...",positive


In [ ]:
df = df[:500]

In [ ]:
(X_train, y_train), (X_test, y_test), preproc = text.texts_from_df(train_df=df, text_column='reviewText', label_columns=['sentiment'], maxlen=100, max_features=100000, preprocess_mode='bert', val_pct=0.1)

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


In [ ]:
model = text.text_classifier(name='bert', train_data=(X_train, y_train), preproc=preproc, metrics=['accuracy'])

Is Multi-Label? False
maxlen is 100
done.


In [ ]:
learner = ktrain.get_learner(model=model, train_data=(X_train, y_train), val_data=(X_test, y_test), batch_size=32, use_multiprocessing=True)

In [ ]:
learner.fit_onecycle(lr=2e-5, epochs=1, checkpoint_folder='folder')



begin training using onecycle policy with max lr of 2e-05...
15/15 [==============================] - 469s 29s/step - loss: 0.3179 - accuracy: 0.9156 - val_loss: 0.2233 - val_accuracy: 0.9400


In [ ]:
predictor = ktrain.get_predictor(learner.model, preproc)

In [ ]:
predictor.predict('I loved the song')

'positive'

# **Building a Document Answering Model**

In [ ]:
from ktrain import text
import os
import shutil

In [ ]:
!wget http://mlg.ucd.ie/files/datasets/bbc-fulltext.zip

--2021-06-04 16:14:25--  http://mlg.ucd.ie/files/datasets/bbc-fulltext.zip
Resolving mlg.ucd.ie (mlg.ucd.ie)... 137.43.93.132
Connecting to mlg.ucd.ie (mlg.ucd.ie)|137.43.93.132|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2874078 (2.7M) [application/zip]
Saving to: ‘bbc-fulltext.zip’

bbc-fulltext.zip    100%[===================>]   2.74M  4.50MB/s    in 0.6s    

2021-06-04 16:14:26 (4.50 MB/s) - ‘bbc-fulltext.zip’ saved [2874078/2874078]



In [ ]:
%%capture
!unzip bbc-fulltext.zip

In [ ]:
from ktrain import text
import os

Change the directory to current BBC folder

In [ ]:
os.chdir(os.getcwd() + '/bbc')

The first step is initializing the index directory. It is used for indexing all the documents. We don't need to create any new directory manually. We just need to pass the name of the index directory to the function initialize_index and then the index directory will be created

In [ ]:
text.SimpleQA.initialize_index('index')

FileIndex(FileStorage('index'), 'MAIN')

After initializing the index, we need to index the documents. Since we will have the all documents in the folder, we will use the function index_from_folder. The index_form_folder takes the folder_path where we will have all the documents and the index_dir as the parameters.

In [ ]:
text.SimpleQA.index_from_folder(folder_path='entertainment', index_dir='index')

The next step is creating an instance of the SimpleQA class shown below. We need to pass the index directory as an argument

In [ ]:
qa = text.SimpleQA('index')

In [ ]:
answers = qa.ask('who who had a global hit with where is the love?')

In [ ]:
qa.display_answers(answers[:5])

,Candidate Answer,Context,Confidence,Document Reference
0,the black eyed peas,"the black eyed peas -who had a global hit with where is the love ?-picked up the prize for best pop act, beating anastacia, avril lavigne, robbie williams and britney spears.",9.997414e-01,153.txt
1,out kast,out kast will add their awards to the four they won at the us mtv awards in august and three grammys in february.,1.829924e-04,153.txt
2,of johnny cash,"he wrote ring of fire with june carter cash, the future wife of johnny cash who went on to score his most popular hit with the track.",7.449709e-05,119.txt
3,s free,"rodgers was singer with early 1970s rocker s free , who had a global hit with all right now, before forming bad company, a successful "" supergroup "" with members of king crimson and mott the hoople.",3.345129e-07,158.txt
4,s free,"rodgers was singer with early 1970s rocker s free , who had a global hit with all right now, before forming bad company, a successful "" supergroup "" with members of king crimson and mott the hoople.",3.345129e-07,138.txt


In [ ]:
answers = qa.ask('who win at mtv europe awards?')

In [ ]:
qa.display_answers(answers[:5])

,Candidate Answer,Context,Confidence,Document Reference
0,duo out kast,us hip hop duo out kast have capped a year of award glory with three prizes at the mtv europe music awards in rome.,0.600309,153.txt
1,was justin timberlake,"last year ' s big winner at the mtv europe awards, held in edinburgh, scotland, was justin timberlake , who walked away with three trophies.",0.157787,153.txt
2,duo out kast,us rap duo out kast ' s trio of trophies at the mtv europe awards crowns a year of huge success for the band.,0.121886,132.txt
3,eminem,"eminem performed on hms belfast on friday, which is docked on the river thames, where he filmed two songs for bbc one ' s top of the pops.",0.059980,151.txt
4,the black sabbath,""" at the end of it i did not like having cameras around the house all the time, "" the black sabbath singer told reporters at the mtv europe awards in rome.",0.059523,206.txt


# **Document Summarization**

In [ ]:
%%capture
!pip install wikipedia==1.4.0
!pip install ktrain==0.25.3

In [ ]:
import wikipedia

In [ ]:
wiki = wikipedia.page('Pablo Picasso')

In [ ]:
doc = wiki.content

In [ ]:
print(doc[:1000])

Pablo Ruiz Picasso (25 October 1881 – 8 April 1973) was a Spanish painter, sculptor, printmaker, ceramicist and theatre designer who spent most of his adult life in France. Regarded as one of the most influential artists of the 20th century, he is known for co-founding the Cubist movement, the invention of constructed sculpture, the co-invention of collage, and for the wide variety of styles that he helped develop and explore. Among his most famous works are the proto-Cubist Les Demoiselles d'Avignon (1907), and Guernica (1937), a dramatic portrayal of the bombing of Guernica by German and Italian air forces during the Spanish Civil War.
Picasso demonstrated extraordinary artistic talent in his early years, painting in a naturalistic manner through his childhood and adolescence. During the first decade of the 20th century, his style changed as he experimented with different theories, techniques, and ideas. After 1906, the Fauvist work of the slightly older artist Henri Matisse motivate

In [ ]:
from ktrain import text
ts = text.TransformerSummarizer()

In [ ]:
ts.summarize(doc)

"Pablo Ruiz Picasso (25 October 1881 – 8 April 1973) was a Spanish painter, sculptor, printmaker, ceramicist and theatre designer. He is known for co-founding the Cubist movement, the invention of constructed sculpture, the co-invention of collage, and for the wide variety of styles that he helped develop and explore. Among his most famous works are the proto-Cubist Les Demoiselles d'Avignon (1907), and Guernica (1937)"

# **Computing Sentence Representation using Sentence as a Service**

In [1]:
%%capture
!pip install tensorflow==1.14
!pip install bert-serving-client
!pip install -U bert-serving-server[http]

Requirement already up-to-date: bert-serving-server[http] in /usr/local/lib/python3.7/dist-packages (1.10.0)


In [2]:
!wget https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip

--2021-06-04 20:21:25--  https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.141.128, 74.125.137.128, 142.250.101.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.141.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 407727028 (389M) [application/zip]
Saving to: ‘uncased_L-12_H-768_A-12.zip’

uncased_L-12_H-768_ 100%[===================>] 388.84M   235MB/s    in 1.7s    

2021-06-04 20:21:27 (235 MB/s) - ‘uncased_L-12_H-768_A-12.zip’ saved [407727028/407727028]



In [3]:
!unzip uncased_L-12_H-768_A-12.zip

Archive:  uncased_L-12_H-768_A-12.zip
   creating: uncased_L-12_H-768_A-12/
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.meta  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001  
  inflating: uncased_L-12_H-768_A-12/vocab.txt  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.index  
  inflating: uncased_L-12_H-768_A-12/bert_config.json  


In [4]:
!nohup bert-serving-start -pooling_strategy REDUCE_MEAN -model_dir=./uncased_L-12_H-768_A-12 > out.file 2>&1 &

In [5]:
from bert_serving.client import BertClient

In [6]:
bc = BertClient() 

KeyboardInterrupt: ignored

In [8]:
sentence1 = 'the weather is great today'
sentence2 = 'it looks like today the weather is pretty nice'

In [9]:
sent_rep1 = BertClient().encode([sentence1])
sent_rep2 = BertClient().encode([sentence2])

KeyboardInterrupt: ignored